# 데이터 불러오기

In [1]:
import pandas as pd

train = pd.read_csv("train.csv")
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# 탐색적 데이터 분석
- 목적 : 주요 변수들을 추출
  + 주요 변수를 몇개까지 추출해야 하나요?
  + 예측 서비스를 구현해야 함. UI/UX도 고려해야 함

## 결측치 확인

In [3]:
train.isnull().sum().sort_values(ascending=False).head(10)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageQual        81
GarageFinish      81
GarageType        81
dtype: int64

## 수치형 / 범주형 변수 구분

In [4]:
numeric_feats = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_feats = train.select_dtypes(include=['object']).columns.tolist()
len(numeric_feats + categorical_feats)

81

## 시각화 & 통계 분석
- 이 부분은 생략
- Target : SalePrice

## 결측치가 30% 이상인 변수 제거

In [5]:
missing_ratio = train.isnull().mean()

# 결측치가 30% 이상인 변수들만 추출
drop_cols = missing_ratio[missing_ratio > 0.3].index.tolist()
drop_cols

['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

## 불필요한 변수 제거 (Id)

In [6]:
drop_cols += ['id']

In [7]:
drop_cols

['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'id']

## 상관관계 높은 변수 수출

In [8]:
corr = train[numeric_feats].corr()['SalePrice'].abs().sort_values(ascending=False)
# 상관계수 0.3 이상
# corr

main_numeric = corr[corr > 0.3].index.tolist()
# 도메인 지식 기반 예시, From 탐색적 데이터 분석을 통해서 추출
main_categorical = ['Neighborhood', 'ExterQual', 'KitchenQual', 'BsmtQual', 'GarageType', 'SaleCondition']  # 도메인 지식 기반 예시

## 최종 주요 변수 추출

In [9]:
main_features = list(set(main_numeric + main_categorical) - set(drop_cols))
main_features

['SaleCondition',
 'Fireplaces',
 'GarageArea',
 'OverallQual',
 'FullBath',
 'GarageType',
 'TotalBsmtSF',
 'TotRmsAbvGrd',
 'BsmtQual',
 'MasVnrArea',
 'KitchenQual',
 'SalePrice',
 '2ndFlrSF',
 'LotFrontage',
 'YearBuilt',
 'GrLivArea',
 'GarageYrBlt',
 'Neighborhood',
 'OpenPorchSF',
 'BsmtFinSF1',
 'YearRemodAdd',
 '1stFlrSF',
 'WoodDeckSF',
 'ExterQual',
 'GarageCars']

## 최종 주요 변수 추출
- SalePrice을 제외한 80개 컬럼에서 ==> 25개로 줄임
- 제 생각 : 여전히 많음, 파이널 프로젝트 진핼할 때 독립변수의 갯수는 10개 이하로 줄이십시오.
- 

In [10]:
main_features = list(set(main_numeric + main_categorical) - set(drop_cols))
len(main_features)

25

In [11]:
main_features

['SaleCondition',
 'Fireplaces',
 'GarageArea',
 'OverallQual',
 'FullBath',
 'GarageType',
 'TotalBsmtSF',
 'TotRmsAbvGrd',
 'BsmtQual',
 'MasVnrArea',
 'KitchenQual',
 'SalePrice',
 '2ndFlrSF',
 'LotFrontage',
 'YearBuilt',
 'GrLivArea',
 'GarageYrBlt',
 'Neighborhood',
 'OpenPorchSF',
 'BsmtFinSF1',
 'YearRemodAdd',
 '1stFlrSF',
 'WoodDeckSF',
 'ExterQual',
 'GarageCars']

## 데이터 분리

In [12]:
X = train[main_features]
y = train['SalePrice']

X.shape, y.shape

((1460, 25), (1460,))

In [13]:
numeric_feats = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_feats = X.select_dtypes(include=['object']).columns.tolist()
categorical_feats

['SaleCondition',
 'GarageType',
 'BsmtQual',
 'KitchenQual',
 'Neighborhood',
 'ExterQual']

## Pipeline 만들기 1차

In [14]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np  # 수치 계산을 위한 numpy
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.impute import SimpleImputer  # 결측치 처리
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # 범주형 변수 인코딩 및 수치형 변수 스케일링
from sklearn.compose import ColumnTransformer  # 컬럼별 전처리 파이프라인 구성
from sklearn.pipeline import Pipeline  # 전체 전처리 및 모델링 파이프라인 구성
from sklearn.linear_model import Ridge, Lasso, ElasticNet  # 선형 회귀 모델
from sklearn.metrics import mean_squared_error, make_scorer  # 모델 평가 지표
from sklearn.model_selection import RandomizedSearchCV  # 랜덤 서치를 통한 하이퍼파라미터 튜닝
from skopt import BayesSearchCV  # 베이지안 최적화를 통한 하이퍼파라미터 튜닝
import xgboost as xgb  # XGBoost 모델
import lightgbm as lgb  # LightGBM 모델
import warnings  # 경고 메시지 처리
import joblib  # 모델 저장 및 로드
import os  # 파일 시스템 작업
import json  # JSON 파일 처리

# LightGBM의 불필요한 경고 메시지 무시 설정
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

## 데이터 전처리 파이프라인 구축
- 수치형 변수 전처리 파이프라인 구성
- 범주형 변수 전처리 파이프라인 구성
- 전체 전처리 파이프라인 통합
- 모델 파이프라인 만들기
- 반복문 수행 작업

In [15]:
# numeric_feats / categorical_feats
# 수치형 변수 전처리 파이프라인 구성
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),              # 결측치 평균값으로 대체
    ('scaler', StandardScaler())                                # 표준화 스케일링 적용
])

# 범주형 변수 전처리 파이프라인 구성
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),              # 결측치 최빈값으로 대체
    ('encoder', OneHotEncoder(handle_unknown='ignore'))                  # 원-핫 인코딩 적용
])

# 전처리 파이프라인 통합
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_feats),                 # 수치형 데이터 처리
    ('cat', categorical_transformer, categorical_feats),         # 범주형 데이터 처리
])

In [16]:
# 2. 모델 후보군 정의
models = {
    'Ridge': Ridge(),  # 릿지 회귀
    'Lasso': Lasso(),  # 라쏘 회귀
    'ElasticNet': ElasticNet(),  # 엘라스틱넷 회귀
    'XGBoost': xgb.XGBRegressor(tree_method='hist', random_state=42),  # XGBoost
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1)  # LightGBM
}

# 3. 모델 학습 및 평가 수행
X = train[main_features]  # 특성 데이터
y = train['SalePrice']  # 타겟 변수

results = {}  # 결과 저장 딕셔너리
for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])
        
        # 3.2 모델 학습
        pipe.fit(X, y)
        
        # 3.3 예측 및 성능 평가
        y_pred = pipe.predict(X)  # 예측값 생성
        rmse = np.sqrt(mean_squared_error(y, y_pred))  # RMSE 계산
        print(f'RMSE: {rmse:.4f}')
        
        results[name] = rmse  # 결과 저장
        
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 비교 및 정렬
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])  # 결과 데이터프레임 생성
results_df = results_df.sort_values('RMSE')  # RMSE 기준 오름차순 정렬
print(results_df)


==== Ridge ====
RMSE: 139.0428

==== Lasso ====
RMSE: 22.3682

==== ElasticNet ====
RMSE: 23373.9541

==== XGBoost ====
RMSE: 117.6804

==== LightGBM ====
RMSE: 7452.0031

==== Final Results ====
        Model          RMSE
1       Lasso     22.368219
3     XGBoost    117.680388
0       Ridge    139.042762
4    LightGBM   7452.003087
2  ElasticNet  23373.954051


C:\Users\Seo\Desktop\python\ml_dl\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [17]:
# numeric_feats / categorical_feats
# 수치형 변수 전처리 파이프라인 구성
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),           # 결측치 평균값으로 대체
    ('scaler', StandardScaler())                           # 표준화 스케일링 적용
])

# 범주형 변수 전처리 파이프라인 구성
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),           # 결측치 최빈값으로 대체
    ('encoder', OneHotEncoder(handle_unknown='ignore'))             # 원-핫 인코딩 적용
])

# 전처리 파이프라인 통합
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_feats),                    # 수치형 데이터 처리
    ('cat', categorical_transformer, categorical_feats),            # 범주형 데이터 처리
])

# 2. 모델 후보군 정의
models = {
    'Ridge': Ridge(),  # 릿지 회귀
    'Lasso': Lasso(),  # 라쏘 회귀
    'ElasticNet': ElasticNet(),  # 엘라스틱넷 회귀
    'XGBoost': xgb.XGBRegressor(tree_method='hist', random_state=42),  # XGBoost
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1)  # LightGBM
}

# 3. 모델 학습 및 평가 수행
X = train[main_features]  # 특성 데이터
y = train['SalePrice']  # 타겟 변수

# 3.1 전체 파이프라인 구성 (전처리 + 모델)
pipe = Pipeline([
    ('preprocessor', preprocessor),  # 전처리 단계
    ('reg', xgb.XGBRegressor(tree_method='hist', random_state=42))  # 모델 단계
])

# 3.2 모델 학습
pipe.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Fireplaces', 'GarageArea',
                                                   'OverallQual', 'FullBath',
                                                   'TotalBsmtSF',
                                                   'TotRmsAbvGrd', 'MasVnrArea',
                                                   'SalePrice', '2ndFlrSF',
                                                   'LotFrontage', 'YearBuilt',
                                                   'GrLivArea', 'GarageYrBlt',
                                                   'OpenPorchSF', 'BsmtFinSF1',
                                                   'YearRemo...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, ...))])

# Pipeline 모델 만들기 2차
- 교차 검증 포함
- 좀 더 정교한 모델을 만들기 위해서 교차검증 + 하이퍼 파라미터 튜닝
- 랜덤서치 적용 

In [19]:
# numeric_feats / categorical_feats
# 수치형 변수 전처리 파이프라인 구성
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),           # 결측치 평균값으로 대체
    ('scaler', StandardScaler())                           # 표준화 스케일링 적용
])

# 범주형 변수 전처리 파이프라인 구성
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),           # 결측치 최빈값으로 대체
    ('encoder', OneHotEncoder(handle_unknown='ignore'))             # 원-핫 인코딩 적용
])

# 전처리 파이프라인 통합
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_feats),                    # 수치형 데이터 처리
    ('cat', categorical_transformer, categorical_feats),            # 범주형 데이터 처리
])

# 2. 모델 후보군 정의
models = {
    'Ridge': Ridge(),  # 릿지 회귀
    'Lasso': Lasso(),  # 라쏘 회귀
    'ElasticNet': ElasticNet(),  # 엘라스틱넷 회귀
    'XGBoost': xgb.XGBRegressor(tree_method='hist', random_state=42),  # XGBoost
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1)  # LightGBM
}

# 2.2 하이퍼파라미터 탐색 공간 정의
# 각 모델에 대한 이해도가 높아야 함
# 그냥 기본값으로 활용
# 하이퍼파라미터를 한다해도 성능을 끌어올리기 쉽지않다 = 가성비가 떨어짐
# 정확도 기준 : 성능 90% 이상이면 ok 따라서 가급적이면 하이퍼 파라미터 최소한 사용
# ChatGPT 질문 키워드 => 과대적합이 발생하지 않도록 튜닝 요청

# 2.2 하이퍼파라미터 탐색 공간 정의
param_spaces = {
    'Ridge': {'reg__alpha': [0.1, 1.0, 10.0]},  # 정규화 강도 - 값이 클수록 더 강한 정규화 적용
    'Lasso': {'reg__alpha': [0.1, 1.0, 10.0]},  # 정규화 강도 - 값이 클수록 더 많은 특성이 0이 됨
    'ElasticNet': {
        'reg__alpha': [0.1, 1.0, 10.0],  # 정규화 강도 - 전체 정규화의 강도 조절
        'reg__l1_ratio': [0.2, 0.5, 0.8]  # L1/L2 비율 - 1에 가까울수록 Lasso, 0에 가까울수록 Ridge와 유사
    },
    'XGBoost': {
        'reg__n_estimators': [100, 500, 1000],  # 트리 개수 - 앙상블에 사용될 트리의 수
        'reg__max_depth': [3, 5, 7],  # 트리 최대 깊이 - 깊을수록 복잡한 패턴 학습 가능
        'reg__learning_rate': [0.01, 0.1, 0.3],  # 학습률 - 각 트리의 기여도를 조절
        'reg__subsample': [0.6, 0.8, 1.0],  # 데이터 샘플링 비율 - 과적합 방지를 위한 데이터 샘플링
        'reg__colsample_bytree': [0.6, 0.8, 1.0]  # 특성 샘플링 비율 - 각 트리마다 사용할 특성의 비율
    },
    'LightGBM': {
        'reg__n_estimators': [100, 500, 1000],  # 트리 개수 - 앙상블에 사용될 트리의 수
        'reg__max_depth': [3, 5, 7],  # 트리 최대 깊이 - 깊을수록 복잡한 패턴 학습 가능
        'reg__learning_rate': [0.01, 0.1, 0.3],  # 학습률 - 각 트리의 기여도를 조절
        'reg__subsample': [0.6, 0.8, 1.0],  # 데이터 샘플링 비율 - 과적합 방지를 위한 데이터 샘플링
        'reg__colsample_bytree': [0.6, 0.8, 1.0]  # 특성 샘플링 비율 - 각 트리마다 사용할 특성의 비율
    }
}

# 3. 모델 학습 및 평가 수행
X = train[main_features]  # 특성 데이터
y = train['SalePrice']  # 타겟 변수

In [20]:
# 3. 모델 학습 및 평가
X = train[main_features]  # 특성 데이터
y = train['SalePrice']  # 타겟 변수

results = {}  # 모델별 RMSE 결과 저장
best_models = {}  # 최적의 모델 저장

for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])
        
        # 3.2 랜덤 서치를 통한 하이퍼파라미터 최적화
        search = RandomizedSearchCV(
            pipe,
            param_spaces[name],
            n_iter=10,  # 탐색 횟수
            cv=5,  # 5-fold 교차 검증
            scoring='neg_root_mean_squared_error',  # 평가 지표 (RMSE)
            random_state=42,  # 재현성을 위한 시드
            n_jobs=-1  # 모든 CPU 코어 사용
        )
        
        # 3.3 모델 학습
        search.fit(X, y)
        
        # 3.4 최적의 모델 저장
        best_models[name] = search.best_estimator_
        
        # 3.5 교차 검증 결과 계산
        cv_scores = -search.cv_results_['mean_test_score']  # RMSE 값 (음수로 저장되어 있어 부호 변환)
        rmse = np.mean(cv_scores)  # 평균 RMSE
        rmse_std = np.std(cv_scores)  # RMSE 표준편차
        
        print(f'Best RMSE: {rmse:.4f} (+/- {rmse_std:.4f})')
        print(f'Best parameters: {search.best_params_}')
        
        results[name] = rmse
        
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 분석
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])
results_df = results_df.sort_values('RMSE')  # RMSE 기준 오름차순 정렬
print(results_df)


==== Ridge ====


C:\Users\Seo\Desktop\python\ml_dl\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best RMSE: 664.1694 (+/- 787.0538)
Best parameters: {'reg__alpha': 0.1}

==== Lasso ====


C:\Users\Seo\Desktop\python\ml_dl\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best RMSE: 25.8663 (+/- 14.7243)
Best parameters: {'reg__alpha': 10.0}

==== ElasticNet ====


C:\Users\Seo\Desktop\python\ml_dl\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best RMSE: 23935.4450 (+/- 14943.3882)
Best parameters: {'reg__l1_ratio': 0.8, 'reg__alpha': 0.1}

==== XGBoost ====
Best RMSE: 9454.7657 (+/- 9442.9680)
Best parameters: {'reg__subsample': 0.6, 'reg__n_estimators': 1000, 'reg__max_depth': 5, 'reg__learning_rate': 0.3, 'reg__colsample_bytree': 0.8}

==== LightGBM ====
Best RMSE: 15911.1794 (+/- 6228.1588)
Best parameters: {'reg__subsample': 0.8, 'reg__n_estimators': 500, 'reg__max_depth': 5, 'reg__learning_rate': 0.01, 'reg__colsample_bytree': 1.0}

==== Final Results ====
        Model          RMSE
1       Lasso     25.866251
0       Ridge    664.169352
3     XGBoost   9454.765684
4    LightGBM  15911.179415
2  ElasticNet  23935.445012


# Pipeline 모델 만들기 3차
- 베이지안

In [21]:
from skopt import BayesSearchCV  # scikit-optimize가 설치되어 있어야 함

results = {}  # 모델별 RMSE 결과 저장
best_models = {}  # 최적의 모델 저장

for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),
            ('reg', model)
        ])

        # 베이지안 서치로 하이퍼파라미터 최적화
        search = BayesSearchCV(
            pipe,
            param_spaces[name],
            n_iter=10,  # 탐색 횟수
            cv=5,  # 5-fold 교차 검증
            scoring='neg_root_mean_squared_error',
            random_state=42,
            n_jobs=-1
        )

        # 모델 학습
        search.fit(X, y)

        # 최적의 모델 저장
        best_models[name] = search.best_estimator_

        # 교차 검증 결과 계산
        cv_scores = -search.cv_results_['mean_test_score']  # 부호 변환
        rmse = np.mean(cv_scores)
        rmse_std = np.std(cv_scores)

        print(f'Best RMSE: {rmse:.4f} (+/- {rmse_std:.4f})')
        print(f'Best parameters: {search.best_params_}')

        results[name] = rmse

    except Exception as e:
        print(f"Error: {str(e)}")

# 최종 결과 분석
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])
results_df = results_df.sort_values('RMSE')
print(results_df)



==== Ridge ====
Best RMSE: 968.1313 (+/- 805.8713)
Best parameters: OrderedDict([('reg__alpha', 0.1)])

==== Lasso ====
Best RMSE: 18.3914 (+/- 10.6444)
Best parameters: OrderedDict([('reg__alpha', 10.0)])

==== ElasticNet ====
Best RMSE: 26490.9901 (+/- 13483.2507)
Best parameters: OrderedDict([('reg__alpha', 0.1), ('reg__l1_ratio', 0.8)])

==== XGBoost ====
Best RMSE: 5959.4285 (+/- 2308.9283)
Best parameters: OrderedDict([('reg__colsample_bytree', 1.0), ('reg__learning_rate', 0.01), ('reg__max_depth', 5), ('reg__n_estimators', 1000), ('reg__subsample', 0.8)])

==== LightGBM ====
Best RMSE: 14014.1997 (+/- 947.1479)
Best parameters: OrderedDict([('reg__colsample_bytree', 1.0), ('reg__learning_rate', 0.3), ('reg__max_depth', 3), ('reg__n_estimators', 1000), ('reg__subsample', 1.0)])

==== Final Results ====
        Model          RMSE
1       Lasso     18.391405
0       Ridge    968.131272
3     XGBoost   5959.428513
4    LightGBM  14014.199702
2  ElasticNet  26490.990083
